In [61]:

from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

/home/shahpearl/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [62]:
import os
import time
from datetime import date
import csv
from bs4 import BeautifulSoup


In [115]:
from bs4 import BeautifulSoup
import requests

# url="https://www.amazon.in/Ainclu-Womens-Classic-Layered-Lace-Up/dp/B00K69XHGY/ref=b2b_gw_d_csf_b2b_sccl_1/258-8837396-8497212?pd_rd_w=SvU1u&content-id=amzn1.sym.80118820-a13a-4810-a5d3-0816775952fa&pf_rd_p=80118820-a13a-4810-a5d3-0816775952fa&pf_rd_r=80CBD46BM18R81225Y26&pd_rd_wg=0hL82&pd_rd_r=c516329a-4a79-48da-8432-a3ec36ed9d0f&pd_rd_i=B083QDVK83&psc=1"
# result=requests.get(url)
# doc=BeautifulSoup(result.text,"html.parser")
# prices=doc.find_all(text="₹")
# parent=prices[0].parent
# print(parent)

# #tags=doc.find_all(["p","div",";li"])

# #tags=doc.find_all(text=re.compile("\$"),limit=1)
# #for tag in tags:
#    #print(tag.strip())
from requests_html import HTMLSession
import json
import time
#Python is an “object-oriented programming language.” This means that almost all the code is 
#implemented using a special construct called classes. Programmers use classes to keep related 
#things together. This is done using the keyword “class,” which is a grouping of object-oriented 
#constructs.
#A class is a code template for creating objects. 
#Objects have member variables and have behaviour associated with them.
#An object is created using the constructor of the class. 
#This object will then be called the instance of the class. In Python we create instances in the following manner
#Instance = class(arguments)
class Reviews:
    def __init__(self) -> None:
        self.session=HTMLSession() #this is our object tht we ll use whenever class instance is called
        self.headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36', 'Origin':'https://www.flipkart.com', 'Referer': 'https://www.flipkart.com/'}
        #self.url= f'https://www.amazon.in/Rovars-U2301-Boys-Black-Light/product-reviews/{self.asin}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
        self.url=None
        #self.driver = None
        self.product_name = None
        self.formatted_product_name = None
        self.product_url=None
        self.product_page_url=None
       
        # to get to amazon india website
    # def open_amazon_website(self):
    #     opt=Options()# creates instance for options
    #     opt.add_argument("--disable-infobars")
    #     opt.add_argument("--disable-extensions")
    #     opt.add_argument('--log-level=OFF')
    #     opt.add_experimental_option('excludeSwitches', ['enable-logging'])
    #     url = "https://www.amazon.in/"
    #     self.driver = webdriver.Chrome(ChromeDriverManager(driver_version='114.0.5735.90').install())
    #     # go to that page using driver
    #     self.driver.get(url)
    #     #to load the website
    #     time.sleep(3)
     #https://www.flipkart.com/search?q=PS5+Digital+Standalone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&as-pos=1&as-type=HISTORY   
            
        #creating url to get to product page eg mobile phones,playstation
    def open_all_product_page(self):
        s=input("\n>>Enter the product name")
        self.product_name=s
        self.formatted_product_name=self.product_name.replace(" ","+")
        
        #fits product name in url
        self.product_url = "https://www.flipkart.com/search?q={}&otracker"
        self.product_url = self.product_url.format(self.formatted_product_name)
        
        print(">> Product URL: ", self.product_url)
        

            
        #to get links of all products in the page
    
    def get_product_link(self):
        # product_name=input('\n>>Enter product name')
        # self.product_name=product_name
        # self.formatted_product_name=self.product_name.replace(" ","-")
        result=requests.get(self.product_url, headers=self.headers)
        soup=BeautifulSoup(result.content,"html.parser")
       
        #links=soup.find_all("a",attrs={'class':"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" })
        #links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
        # Get all elements which have an attribute data-id
        elems = soup.find_all(attrs={'data-id': True})
        firstelement=elems[0]
        # Get all links from the first element
        childelement = firstelement.find_all('a')
        # Get the first child link
        firstchild = childelement[0]        
        self.product_page_url= 'https://www.flipkart.com' + firstchild['href']
       
        print(">>Product page URL:",self.product_page_url)    
                
    def get_review_url(self):
        self.url=self.product_page_url.replace("/p/","/product-reviews/")
        print(">>Review page URL:",self.url)
        
    #all reviews from all review pages of one product
    def find_all_reviews(self):
        pageNum = 1
        data = []
        total_average_page_rating=0
        total_average_sentimental_score=0
        while True:
            pageData,avg_rating,avg_sentimental_score = self.parseReviewsPage(str(pageNum))
            if len(pageData) == 0:
                break
            data.extend(pageData)
            total_average_page_rating+=avg_rating
            total_average_sentimental_score+=avg_sentimental_score
            pageNum += 1
            
        return data,total_average_page_rating/(pageNum),total_average_sentimental_score/(pageNum)
    
    
    def parseReviewsPage(self, pageNum):
        reviewResult = requests.get(self.url + '&page=' + pageNum, headers=self.headers)
        reviewSoup = BeautifulSoup(reviewResult.content, 'html.parser')

        bigBox = reviewSoup.find_all('div', attrs={'class': '_1YokD2 _3Mn1Gg col-9-12'})[0]

        # All divs in the bigBox with class name as '_1AtVbE col-12-12'
        reviewBoxes = bigBox.find_all('div', attrs={'class': '_1AtVbE col-12-12'})
        data = []
        rating_count=0   
        ratings=0  
        avg_rating=0 
        total_sentimental_score=0
        for reviewBox in reviewBoxes:
            # check if reviewbox has a div with class name as 'row'
            rows = reviewBox.find_all('div', attrs={'class': 'row'})
            if len(rows) >= 2:
                finalRows = rows[0:2]
                row1 = finalRows[0]
                row2 = finalRows[1]
                # child div of row1 with class _3LWZlK
                ratingDiv = row1.find('div', attrs={'class': '_3LWZlK'})
                if ratingDiv is None:
                    continue
                rating = ratingDiv.text
                # child p of row1 
                reviewTitle = row1.find('p')
                title=""
                # if reviewTitle is None:
                #     continue
                if reviewTitle is not None:
                   title = reviewTitle.text
                # child div of row2 with class ""
                reviewBody = row2.find('div', attrs={'class': ''})
                body=""
                if reviewBody is not None:
                    body=reviewBody.text
                
                # if reviewBody is None:
                #     continue
               
                body = body.replace('\n', '').strip().replace('READ MORE', '')
                rating_count=rating_count+1
                ratings=ratings+(int(rating[0]))
                sentiment= sentiment_pipeline(body)[0]
                if sentiment['label']=='POSITIVE':
                    total_sentimental_score=sentiment['score']+total_sentimental_score
                else:
                    total_sentimental_score=total_sentimental_score-sentiment['score']
                data.append({
                    'Rating': int(rating),
                    'Title': title,
                    'Body': body,
                    'sentiment_score':sentiment['score'],
                    'sentiment_label':sentiment['label']
                })
                
        avg_rating=ratings/len(reviewBoxes)
        avg_sentimental_score=total_sentimental_score/len(reviewBoxes)       
        return data,avg_rating,avg_sentimental_score


        
        
          

   # review parsing
    def parse(self,all_ratings,all_body):
        total=[]  
        rating_count=0   
        ratings=0  
        avg_rating=0 
        total_sentimental_score=0
        for i in range(1,len(all_ratings)):
            #title=review.find('a[data-hook=review-title]',first=True).text
            rating=all_ratings[i]
            body=all_body[i].replace('\n','').strip()
            rating_count=rating_count+1
            ratings=ratings+(int(rating[0]))
            sentiment= sentiment_pipeline(body)[0]
            if sentiment['label']=='POSITIVE':
               total_sentimental_score=sentiment['score']+total_sentimental_score
            else:
               total_sentimental_score=total_sentimental_score-sentiment['score']
            
            data={
                'rating_count':rating_count,
               
                'rating':rating,
                'body':body[:100],
                'sentiment':sentiment
            }
            total.append(data)
        avg_rating=ratings/len(all_ratings)
        avg_sentimental_score=total_sentimental_score/rating_count
        return total,avg_rating,avg_sentimental_score
    
    
    
    def save(self,results):
        with open(self.product_name+'reviews.json','w') as f:
            json.dump(results,f)
#https://www.amazon.in/Sony-PS5-PlayStation-Console/dp/B0BRCP72X8
            
            
            
amz=Reviews()

amz.open_all_product_page()

amz.get_product_link()
amz.get_review_url()
results=[]
average_rating=0

    
time.sleep(0.3)
reviewData,average_rating,avg_sentimental_score=amz.find_all_reviews()

print(reviewData)
# results.append(amz.parse(ratings,body))
# temp,average_rating,avg_sentimental_score=amz.parse(ratings,body)
    


amz.save(results)         



>> Product URL:  https://www.flipkart.com/search?q=Playstation+5&otracker
>>Product page URL: https://www.flipkart.com/sony-playstation-5-digital-825-gb-astro-s-playroom/p/itm3c6e8c91e0941?pid=GMCFYTWSM9Q9SN3C&lid=LSTGMCFYTWSM9Q9SN3CKRLRPQ&marketplace=FLIPKART&q=Playstation+5&store=4rr%2Fx1m%2Fogz&spotlightTagId=TrendingId_4rr%2Fx1m%2Fogz&srno=s_1_1&otracker=search&fm=organic&iid=5d63adcd-8718-4aa3-abb7-3c61d4c9ab98.GMCFYTWSM9Q9SN3C.SEARCH&ppt=None&ppn=None&ssid=2qnlfn87j40000001703883419482&qH=190ad89e15ef902c
>>Review page URL: https://www.flipkart.com/sony-playstation-5-digital-825-gb-astro-s-playroom/product-reviews/itm3c6e8c91e0941?pid=GMCFYTWSM9Q9SN3C&lid=LSTGMCFYTWSM9Q9SN3CKRLRPQ&marketplace=FLIPKART&q=Playstation+5&store=4rr%2Fx1m%2Fogz&spotlightTagId=TrendingId_4rr%2Fx1m%2Fogz&srno=s_1_1&otracker=search&fm=organic&iid=5d63adcd-8718-4aa3-abb7-3c61d4c9ab98.GMCFYTWSM9Q9SN3C.SEARCH&ppt=None&ppn=None&ssid=2qnlfn87j40000001703883419482&qH=190ad89e15ef902c
[{'Rating': 5, 'Title': 'Su

In [116]:
import pandas as pd

In [117]:
df = pd.DataFrame(reviewData)
Total = (df['Rating']).sum()/len(df)
print(Total)
print(f"The average rating is: {Total}\n")
sentiment_total=df[df["sentiment_label"] == 'POSITIVE']['sentiment_score'].sum() - df[df["sentiment_label"] == 'NEGATIVE']['sentiment_score'].sum()
sentiment_total=sentiment_total/len(df)
print(f"The average sentimental score is: {sentiment_total}")
if sentiment_total>0:
    print("Overall sentimental review is POSITIVE")
else:
    print("Overall sentimental review is NEGATIVE")


df

4.7725118483412325
The average rating is: 4.7725118483412325

The average sentimental score is: 0.7699832724168967
Overall sentimental review is POSITIVE


,Rating,Title,Body,sentiment_score,sentiment_label
0,5,Super!,This is a beast. Digital edition looks so good...,0.999671,POSITIVE
1,5,Fabulous!,"Finally, after 4 months i grabbed one. Product...",0.999398,POSITIVE
2,5,Just wow!,The best console in the world right now wrt ex...,0.998157,POSITIVE
3,5,Excellent,After waiting a year i got this gaming beast. ...,0.999849,POSITIVE
4,5,Best in the market!,Worth every penny!,0.999690,POSITIVE
...,...,...,...,...,...
206,5,Best in the market!,Good experience so far.. Packing was good n ha...,0.995165,POSITIVE
207,5,Worth every penny,Its good 😎,0.999843,POSITIVE
208,5,Great product,Awesome.. Must Have Product For Gamers Dualsen...,0.999127,POSITIVE
209,5,Brilliant,Awesome Product 😌,0.999861,POSITIVE


In [77]:
# sort by rating


KeyError: 'rating'